# Web Scraping From SET

setting to authorize this chromdrive can be opened

command : xattr -d com.apple.quarantine \<name-of-executable\>

In [2]:
!xattr -d com.apple.quarantine chromedriver

xattr: chromedriver: No such xattr: com.apple.quarantine


## Web scraping

In [3]:
import pandas as pd
from selenium import  webdriver
import os

### Connect to chrome

In [4]:
driver = webdriver.Chrome(os.getcwd() + "/chromedriver")
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="53d922a8bd1efbeb0911de3aeffc15cf")>

### Collect all company names from SET

In [5]:
def get_all_company_names() : 
    list_page = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", "NUMBER"]
    df_all_com_names = pd.DataFrame()

    # open set.or.th to collect comppany names into dataframe
    for p in list_page :
        driver.get("https://classic.set.or.th/set/commonslookup.do?language=th&country=TH&prefix=" + p)
        page = driver.page_source #HTML
        df_page = pd.read_html(page)[0] #HTML to Dataframe
        df_all_com_names = df_all_com_names.append(df_page)
    
    # transform
    df_all_com_names.set_index("ชื่อย่อหลักทรัพย์", inplace=True)

    # return dataframe
    return df_all_com_names

    

In [6]:
df_all_com_names = get_all_company_names()

check company names

In [7]:
df_all_com_names.index

Index(['A', 'A5', 'AAV', 'ABICO', 'ABM', 'ABPIF', 'ACAP', 'ACC', 'ACE', 'ACG',
       ...
       'WPH', 'XO', 'XPG', 'YGG', 'YUASA', 'ZEN', 'ZIGA', '2S', '3K-BAT',
       '7UP'],
      dtype='object', name='ชื่อย่อหลักทรัพย์', length=852)

In [9]:
df_all_com_names

,ชื่อเต็มบริษัท / หลักทรัพย์จดทะเบียน,ตลาด
ชื่อย่อหลักทรัพย์,,
A,บริษัท อารียา พรอพเพอร์ตี้ จำกัด (มหาชน),SET
A5,บริษัท แอสเซท ไฟว์ กรุ๊ป จำกัด (มหาชน),mai
AAV,บริษัท เอเชีย เอวิเอชั่น จำกัด (มหาชน),SET
ABICO,บริษัท เอบิโก้ โฮลดิ้งส์ จำกัด (มหาชน),mai
ABM,บริษัท เอเชีย ไบโอแมส จำกัด (มหาชน),mai
...,...,...
ZEN,บริษัท เซ็น คอร์ปอเรชั่น กรุ๊ป จำกัด (มหาชน),SET
ZIGA,บริษัท ซิก้า อินโนเวชั่น จำกัด (มหาชน),mai
2S,บริษัท 2 เอส เมทัล จำกัด (มหาชน),SET


### Collect stock information of each company

In [12]:
# this symbols are difference format. it's stock fund.
format_info_error = ["CPNCG", "CPTGF", "ERWPF", "KPNPF", "LPF", "M-II", "M-STOR", "MNIT2", "POPF", "PPF", "SIRIP", "SSPF", "TLHPF", "TTLPF"]

df_com_names = df_all_com_names.drop(index=format_info_error)

In [15]:
def get_stock_information() :   
    
    import time
    start = time.time()
    print("Start!!")
    
    stock_information = pd.DataFrame()
    stocks = df_com_names.index
    
    for stock in stocks :
        if stock == "F&D" : 
            stock = "F%26D"
        elif stock == "L&E" :
            stock = "L%26E"
            
        url = f"https://classic.set.or.th/set/factsheet.do?symbol={stock}&ssoPageId=3&language=th&country=TH"
        driver.get(url)
        factsheet = driver.page_source
        
        # as = alias (stock symbol)
        as_df = pd.read_html(factsheet)[2]
        as_df.drop(columns=1, inplace=True)
        as_df.columns = ["ชื่อย่อหลักทรัพย์"]
        
        if pd.read_html(factsheet, index_col=0)[9].iloc[0][1] == "ข้อมูลเบื้องต้น" :
            # a_df = information
            a_df = pd.read_html(factsheet, index_col=0)[9]
            a_df = a_df.transpose()
            a_df = a_df.iloc[[0]]
            a_df = a_df[["ที่อยู่", "เบอร์โทรศัพท์", "เบอร์โทรสาร", "URL", "พาร์"]]
            a_df.reset_index(drop=True, inplace=True)
        
        elif pd.read_html(factsheet, index_col=0)[9].iloc[0][1] == "ข่าวล่าสุด" :
            # a_df = information
            a_df = pd.read_html(factsheet, index_col=0)[10]
            a_df = a_df.transpose()
            a_df = a_df.iloc[[0]]
            a_df = a_df[["ที่อยู่", "เบอร์โทรศัพท์", "เบอร์โทรสาร", "URL", "พาร์"]]
            a_df.reset_index(drop=True, inplace=True)
        

        # b_df = industrial / category
        b_df = pd.read_html(factsheet)[5]
        b_df = b_df[0].str.split(r"/", expand=True)
        b_df.rename(columns={0 : "กลุ่มอุตสาหกรรม", 1 : "หมวดธุรกิจ"}, inplace=True)
        b_df = b_df.iloc[[0]]
        
        
        if pd.read_html(factsheet, index_col=0)[18].iloc[0][1] == "สะสม: อัตราหมุนเวียนปริมาณการซื้อขาย, มูลค่าซื้อขาย/วัน" :
            # c_df = statistical
            c_df = pd.read_html(factsheet, index_col=0)[18]
            c_df = c_df.transpose()
            c_df.dropna(axis = 1, how = 'all', inplace=True)
            c_df = c_df.iloc[[0]]
            c_df.drop(columns=["สะสม: อัตราหมุนเวียนปริมาณการซื้อขาย, มูลค่าซื้อขาย/วัน", "ข้อมูลสถิติ", "อัตราผลตอบแทน"], inplace=True)
            c_df.reset_index(drop=True, inplace=True)
            
            # d_bg_df = biger shareholder
            d_df = pd.read_html(factsheet)[16]
            d_df.drop(columns=0, inplace=True)
            d_df.columns = [d_df[1][0], d_df[2][0], d_df[3][0]]
            d_df.drop(index=0, inplace=True)
            d_df.reset_index(drop=True, inplace=True)
            d_bg_df = d_df.iloc[[0]]
            d_bg_df.columns = ["ผู้ถือหุ้นรายใหญ่", "จำนวนหุ้น", "สัดส่วนหุ้น"]

        
        elif pd.read_html(factsheet, index_col=0)[18].iloc[0][1] == "คณะกรรมการ" :
            # c_df = statistical
            c_df = pd.read_html(factsheet, index_col=0)[19]
            c_df = c_df.transpose()
            c_df.dropna(axis = 1, how = 'all', inplace=True)
            c_df = c_df.iloc[[0]]
            c_df.drop(columns=["สะสม: อัตราหมุนเวียนปริมาณการซื้อขาย, มูลค่าซื้อขาย/วัน", "ข้อมูลสถิติ", "อัตราผลตอบแทน"], inplace=True)
            c_df.reset_index(drop=True, inplace=True)

            # d_bg_df = biger shareholder
            d_df = pd.read_html(factsheet)[17]
            d_df.drop(columns=0, inplace=True)
            d_df.columns = [d_df[1][0], d_df[2][0], d_df[3][0]]
            d_df.drop(index=0, inplace=True)
            d_df.reset_index(drop=True, inplace=True)
            d_bg_df = d_df.iloc[[0]]
            d_bg_df.columns = ["ผู้ถือหุ้นรายใหญ่", "จำนวนหุ้น", "สัดส่วนหุ้น"]
            
    
        elif pd.read_html(factsheet, index_col=0)[18].iloc[0][1] == "การจ่ายปันผล" :
            # c_df = statistical
            c_df = pd.read_html(factsheet, index_col=0)[17]
            c_df = c_df.transpose()
            c_df.dropna(axis = 1, how = 'all', inplace=True)
            c_df = c_df.iloc[[0]]
            c_df.drop(columns=["สะสม: อัตราหมุนเวียนปริมาณการซื้อขาย, มูลค่าซื้อขาย/วัน", "ข้อมูลสถิติ", "อัตราผลตอบแทน"], inplace=True)
            c_df.reset_index(drop=True, inplace=True)

            # d_bg_df = biger shareholder
            d_df = pd.read_html(factsheet)[15]
            d_df.drop(columns=0, inplace=True)
            d_df.columns = [d_df[1][0], d_df[2][0], d_df[3][0]]
            d_df.drop(index=0, inplace=True)
            d_df.reset_index(drop=True, inplace=True)
            d_bg_df = d_df.iloc[[0]]
            d_bg_df.columns = ["ผู้ถือหุ้นรายใหญ่", "จำนวนหุ้น", "สัดส่วนหุ้น"]
            
            
        elif pd.read_html(factsheet, index_col=0)[18].iloc[0][1] == "นางสาว กมลทิพย์ เลิศวิทย์วรเทพ/บริษัท สำนักงาน อีวาย จำกัด  นาย วิชาติ โลเกศกระวี/บริษัท สำนักงาน อีวาย จำกัด  นางสาว สาธิดา รัตนานุรักษ์/บริษัท สำนักงาน อีวาย จำกัด  นางสาว ศิริวรรณ นิตย์ดำรง/บริษัท สำนักงาน อีวาย จำกัด" :
            # c_df = statistical
            c_df = pd.read_html(factsheet, index_col=0)[16]
            c_df = c_df.transpose()
            c_df.dropna(axis = 1, how = 'all', inplace=True)
            c_df = c_df.iloc[[0]]
            c_df.drop(columns=["สะสม: อัตราหมุนเวียนปริมาณการซื้อขาย, มูลค่าซื้อขาย/วัน", "ข้อมูลสถิติ", "อัตราผลตอบแทน"], inplace=True)
            c_df.reset_index(drop=True, inplace=True)

            # d_bg_df = biger shareholder
            d_df = pd.read_html(factsheet)[15]
            d_df.drop(columns=0, inplace=True)
            d_df.columns = [d_df[1][0], d_df[2][0], d_df[3][0]]
            d_df.drop(index=0, inplace=True)
            d_df.reset_index(drop=True, inplace=True)
            d_bg_df = d_df.iloc[[0]]
            d_bg_df.columns = ["ผู้ถือหุ้นรายใหญ่", "จำนวนหุ้น", "สัดส่วนหุ้น"]

        
        # e_df = financail highlight
        driver.get(f"https://classic.set.or.th/set/companyhighlight.do?symbol={stock}&language=th&country=TH")
        finance = driver.page_source
        e_df = pd.read_html(finance,index_col=0)[0]
        e_df = e_df.iloc[1:9]
        e_df = e_df.transpose()
        e_df = e_df.iloc[[-1]]
        e_df.reset_index(drop=True, inplace=True)
        
        
        # join dataframe
        info = as_df.join([b_df, a_df, c_df, d_bg_df, e_df])

        # append to stock_information
        stock_information = stock_information.append(info)
         
    end = time.time()
    print("Done!!!", end - start, "second")
        
    return stock_information


In [16]:
stock_information = get_stock_information()

Start!!
Done!!! 3393.516547203064 second


In [24]:
stock_information.reset_index(drop=True, inplace=True)

### Output

In [25]:
stock_information.head()

,ชื่อย่อหลักทรัพย์,กลุ่มอุตสาหกรรม,หมวดธุรกิจ,ที่อยู่,เบอร์โทรศัพท์,เบอร์โทรสาร,URL,พาร์,จำนวนหุ้นจดทะเบียน (ล้านหุ้น),มูลค่าหลักทรัพย์ตามราคาตลาด (ลบ.),...,มูลค่าหุ้นที่เรียกชำระแล้ว,รายได้รวม,กำไร (ขาดทุน) จากกิจกรรมอื่น,กำไรสุทธิ,กำไรต่อหุ้น (บาท),มูลค่าหน่วยลงทุน/หน่วย (บาท),2,3,4,อัตราส่วนทางการเงินที่สำคัญ
0,A,อสังหาริมทรัพย์และก่อสร้าง,พัฒนาอสังหาริมทรัพย์,999 ถนนประดิษฐ์มนูธรรม แขวงสะพานสอง เขตวังทองห...,"0-2798-9999, 1797",0-2798-9990,http://www.areeya.co.th,1.00 บาท,980.00,4998.00,...,980.00,965.50,-,32.72,0.03,NaN,NaN,NaN,NaN,NaN
1,A5,อสังหาริมทรัพย์และก่อสร้าง,,เลขที่ 349 อาคารเอสเจ อินฟินิท วัน บิสซิเนส คอ...,0-2026-3512,0-2026-3513,http://www.assetfive.co.th,0.50 บาท,1209.38,3241.15,...,604.69,322.04,-1.34,50.08,0.04,NaN,NaN,NaN,NaN,NaN
2,AAV,บริการ,ขนส่งและโลจิสติกส์,222 ท่าอากาศยานดอนเมืองอาคารส่วนกลางชั้น 3 ห้อ...,0-2562-5700,0-2562-5705,http://www.aavplc.com,0.10 บาท,11592.86,32923.71,...,1159.29,2097.22,-,-2370.60,-0.21,NaN,NaN,NaN,NaN,NaN
3,ABICO,เกษตรและอุตสาหกรรมอาหาร,,401/1หมู่ที่ 8 อาคารเอบิโก้ ชั้น 5 ถนนพหลโยธิน...,0-2080-7899,0-2080-7805,http://www.abicogroup.com,1.00 บาท,244.40,1918.52,...,244.40,654.33,-0.79,26.17,0.11,NaN,NaN,NaN,NaN,NaN
4,ABM,ทรัพยากร,,273/2 ถนนพระรามที่ 2 แขวงแสมดำ เขตบางขุนเทียน ...,0-2415-0054,0-2415-1102,http://www.asiabiomass.com,0.50 บาท,300.00,852.00,...,150.00,612.90,-,6.75,0.02,NaN,NaN,NaN,NaN,NaN


In [27]:
drop = ["มูลค่าหน่วยลงทุน/หน่วย (บาท)",2, 3, 4, "อัตราส่วนทางการเงินที่สำคัญ"]

stock_information.drop(columns=drop, inplace=True)

In [28]:
stock_information.head()

,ชื่อย่อหลักทรัพย์,กลุ่มอุตสาหกรรม,หมวดธุรกิจ,ที่อยู่,เบอร์โทรศัพท์,เบอร์โทรสาร,URL,พาร์,จำนวนหุ้นจดทะเบียน (ล้านหุ้น),มูลค่าหลักทรัพย์ตามราคาตลาด (ลบ.),...,จำนวนหุ้น,สัดส่วนหุ้น,สินทรัพย์รวม,หนี้สินรวม,ส่วนของผู้ถือหุ้น,มูลค่าหุ้นที่เรียกชำระแล้ว,รายได้รวม,กำไร (ขาดทุน) จากกิจกรรมอื่น,กำไรสุทธิ,กำไรต่อหุ้น (บาท)
0,A,อสังหาริมทรัพย์และก่อสร้าง,พัฒนาอสังหาริมทรัพย์,999 ถนนประดิษฐ์มนูธรรม แขวงสะพานสอง เขตวังทองห...,"0-2798-9999, 1797",0-2798-9990,http://www.areeya.co.th,1.00 บาท,980.00,4998.00,...,320897000,32.74,12913.99,9775.88,3138.11,980.00,965.50,-,32.72,0.03
1,A5,อสังหาริมทรัพย์และก่อสร้าง,,เลขที่ 349 อาคารเอสเจ อินฟินิท วัน บิสซิเนส คอ...,0-2026-3512,0-2026-3513,http://www.assetfive.co.th,0.50 บาท,1209.38,3241.15,...,536801385,44.39,1611.90,732.85,859.71,604.69,322.04,-1.34,50.08,0.04
2,AAV,บริการ,ขนส่งและโลจิสติกส์,222 ท่าอากาศยานดอนเมืองอาคารส่วนกลางชั้น 3 ห้อ...,0-2562-5700,0-2562-5705,http://www.aavplc.com,0.10 บาท,11592.86,32923.71,...,2973616671,25.65,67589.49,54100.92,13488.58,1159.29,2097.22,-,-2370.60,-0.21
3,ABICO,เกษตรและอุตสาหกรรมอาหาร,,401/1หมู่ที่ 8 อาคารเอบิโก้ ชั้น 5 ถนนพหลโยธิน...,0-2080-7899,0-2080-7805,http://www.abicogroup.com,1.00 บาท,244.40,1918.52,...,61015364,24.97,2055.03,1037.36,1017.67,244.40,654.33,-0.79,26.17,0.11
4,ABM,ทรัพยากร,,273/2 ถนนพระรามที่ 2 แขวงแสมดำ เขตบางขุนเทียน ...,0-2415-0054,0-2415-1102,http://www.asiabiomass.com,0.50 บาท,300.00,852.00,...,58529805,19.51,951.25,585.89,382.78,150.00,612.90,-,6.75,0.02


In [29]:
stock_information.to_csv(os.getcwd() + "/Output/company_names.csv", index=False, encoding="utf-8")